In [ ]:
import sc3nb as scn

In [ ]:
sc = scn.startup()

In [ ]:
buffer = sc.Buffer().load_file("./my_recording.wav")

# Synth Definition
You can create a pseudo synth definition with the SynthDef subclass in sc:

In [ ]:
synthDef = sc.SynthDef(name="synthDef", definition="""
    { |out=0, bufnum=0, rate=1, loop=0, pan=0, amp=0.3 |
        Out.ar(out,
        PlayBuf.ar({{channel}}, bufnum, rate*BufRateScale.kr(bufnum), 
            loop: ^loop, 
            doneAction: Done.freeSelf)!2
    )}""")

Here we have a new definition with name "synthDef" and dynamic content "channel". Now we want to replace the dynamic channel to a channel amount:

In [ ]:
synthDef.set_context("channel", 2)
synthDef.current_def

The definition is now complete -> lets create it in SC:
You can also add pyvars here if you want to modify the rendered query at the end.

In [ ]:
name = synthDef.create(pyvars={"loop": 0})
name

Delete an existing synthdef in sc (don't execute this, when you want to use this synthdef in the synth examples below)

In [ ]:
# synthDef.free(name)

Remove all unused placeholder from current_def

In [ ]:
synthDef.reset()
synthDef.unset_remaining()
synthDef.current_def

Here you see, that the placeholder {{channel}} is deleted. With this method you make sure, that you don't have any unused placeholders in the definition before create a SynthDef.

Use new created synthDef:

## Synth
Create a new synth:

In [ ]:
synth = sc.Synth(name=name, args={"bufnum": buffer.bufnum})

Restart the synth

In [ ]:
synth.restart()

Pause a synth (restart it again, because the synth was already finished)

In [ ]:
synth.restart().pause()

Run a paused synth:

In [ ]:
synth.run()

Set a control to a synth (restart & pause it before, when it is a short, self destroying synth)

In [ ]:
synth.set("loop", 1)

Hint: You don't have to restart your synth manually, if you have an already running synth. 

In [ ]:
synth.restart()

Destroy a running synth:

In [ ]:
synth.free()

## Example creation of many SynthDefs
In some cases you want to create many SynthDefs with only a small change. You can use the SynthDefs object multiple time to do this. Here we want to create playbuf synthdefs for 1 to 10 channels:
(Reuse of the synthdef object, which is defined above)

In [ ]:
synthPlaybufs = {}
for channel in [1,2,3,4,5,6,7,8,9,10]:
    synthPlaybufs[channel] = synthDef.reset().set_context(
        "channel", channel).create()

Now you can access via ``synthPlayBufs[2]`` to the 2-ch playbuf etc.